**HI ALL**

# Connecting to storage System

In [21]:
# from google.colab import drive
# drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [22]:
# ls

 Colab_Sim_CHRIS.ipynb   README.md                        'test2_!_local.py'   URDFs/
 joints.txt              test1_agent+box+arm_movement.py   test2_.py           vid.mp4


In [23]:
# cd /content/gdrive/MyDrive/FIFA 25/PyBullet

/content/gdrive/MyDrive/FIFA 25/PyBullet


In [24]:
!ls

 Colab_Sim_CHRIS.ipynb	 README.md			  'test2_!_local.py'   URDFs
 joints.txt		 test1_agent+box+arm_movement.py   test2_.py	       vid.mp4


In [25]:
!nvidia-smi


Sun Sep 15 04:14:17 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0              42W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [26]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Sep 15 04:14:18 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0              42W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [27]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 89.6 gigabytes of available RAM

You are using a high-RAM runtime!


# Installing packages

Install all the used packages

In [28]:
# !pip install stable-baselines3
# !pip install --upgrade tensorflow
!pip install pybullet imageio-ffmpeg

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 MB 18.9 MB/s eta 0:00:00


In [29]:
# !pip install cloudpickle==3.0.0
# !pip install gym==0.26.2
# !pip install gym-notices==0.0.8
# !pip install numpy==2.0.1
# !pip install pybullet==3.2.6

Check the versions to make sure all is compatible

In [30]:
import stable_baselines3 as sb
import tensorflow as tf
import pkg_resources
import imageio_ffmpeg
print('stable_baselines3 Version:', sb.__version__)
print('tensorflow Version:', tf.__version__)
print('pybullet Version:', pkg_resources.get_distribution("pybullet").version)
print('imageio_ffmpeg Version:', imageio_ffmpeg.__version__)

# THE OUTPUT OF THE PRINT STATMENTS
# stable_baselines3 Version: 2.3.2
# tensorflow Version: 2.17.0
# pybullet Version: 3.2.6
# imageio_ffmpeg Version: 0.5.1

stable_baselines3 Version: 2.3.2
tensorflow Version: 2.17.0
pybullet Version: 3.2.6
imageio_ffmpeg Version: 0.5.1


# Creating a base code

In [31]:
import os
import time
import math
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import pybullet as p
import pybullet_data
import cv2
import imageio_ffmpeg
from base64 import b64encode
from IPython.display import HTML



In [32]:
# Optimized Colab class for managing video and frames
class Colab():
    def __init__(self):
        # Camera parameters
        self.cam_target_pos = [0, 0, 0]
        self.cam_distance = 5.0
        self.cam_yaw, self.cam_pitch, self.cam_roll = 50, -30.0, 0
        self.cam_width, self.cam_height = 480, 368
        self.cam_up, self.cam_up_axis_idx, self.cam_near_plane, self.cam_far_plane, self.cam_fov = [0, 0, 1], 2, 0.01, 100, 60

        # Set up video writer
        self.vid = imageio_ffmpeg.write_frames('vid.mp4', (self.cam_width, self.cam_height), fps=30)
        self.vid.send(None)  # Seed the video writer with a blank frame

    def playVideo(self):
        # Create a video player for the generated video
        self.mp4 = open('vid.mp4', 'rb').read()
        self.data_url = "data:video/mp4;base64," + b64encode(self.mp4).decode()
        video = HTML('<video width=480 controls><source src="%s" type="video/mp4"></video>' % self.data_url)
        return video

    def addFrame(self):
        # Capture frame for video
        cam_view_matrix = p.computeViewMatrixFromYawPitchRoll(self.cam_target_pos, self.cam_distance, self.cam_yaw, self.cam_pitch, self.cam_roll, self.cam_up_axis_idx)
        cam_projection_matrix = p.computeProjectionMatrixFOV(self.cam_fov, self.cam_width / self.cam_height, self.cam_near_plane, self.cam_far_plane)
        image = p.getCameraImage(self.cam_width, self.cam_height, cam_view_matrix, cam_projection_matrix)[2][:, :, :3]
        self.vid.send(np.ascontiguousarray(image))

# Optimized Simulation class for handling the physics simulation

In [33]:
class Simulation(Colab):
    def __init__(self):
        super().__init__()
        # Use DIRECT mode for faster simulation without GUI
        self.physics_client = p.connect(p.DIRECT)
        p.setAdditionalSearchPath(pybullet_data.getDataPath())
        p.resetSimulation()
        p.setGravity(0, 0, -9.81)

        self.env = Environment()
        self.agent = Agent()
        self.time_step = 1. / 240.

        self.loop()

    def loop(self):
        # Main simulation loop
        for t in range(200):
            print(f'\rtimestep {t+1}...', end='')
            if t % 8 == 0:  # Capture every 8th frame
                self.addFrame()
            self.agent.moveJoint(1)
            p.stepSimulation()
        self.vid.close()

In [34]:
import pybullet as p
import pybullet_data
import numpy as np
import time

class Environment:
    def __init__(self):
        self.loadEnvironment()

    def loadEnvironment(self):
        # Set gravity and data path for PyBullet assets
        p.setAdditionalSearchPath(pybullet_data.getDataPath())
        p.setGravity(0, 0, -9.81)

        # Load a plane to represent the field ground
        plane_id = p.loadURDF("plane.urdf", basePosition=[0, 0, 0])
        p.changeVisualShape(plane_id, -1, rgbaColor=[0.1, 0.5, 0.1, 1])  # Green field color

        # Define the field size
        field_length = 100  # Length of the field (in meters)
        field_width = 60    # Width of the field (in meters)

        # Add field boundaries (white lines)
        line_thickness = 0.1  # Slightly increase thickness for better visibility

        # Create boundary lines
        self.add_line([-field_length/2, -field_width/2], [-field_length/2, field_width/2], line_thickness)  # Left sideline
        self.add_line([field_length/2, -field_width/2], [field_length/2, field_width/2], line_thickness)    # Right sideline
        self.add_line([-field_length/2, field_width/2], [-field_length/2, -field_width/2], line_thickness)   # Bottom goal line
        self.add_line([field_length/2, field_width/2], [-field_length/2, field_width/2], line_thickness)    # Top goal line

        # Center circle
        self.add_circle([0, 0], radius=9.15, line_thickness=line_thickness)  # Standard center circle radius

        # Add goals at both ends
        self.create_goal([-field_length/2, 0], goal_width=7.32, goal_height=2.44)  # Left goal
        self.create_goal([field_length/2, 0], goal_width=7.32, goal_height=2.44)   # Right goal

    def add_line(self, start_pos, end_pos, thickness):
        # Calculate line length and orientation
        length = np.linalg.norm(np.array(end_pos) - np.array(start_pos))
        orientation = np.arctan2(end_pos[1] - start_pos[1], end_pos[0] - start_pos[0])

        # Create a box shape to represent the line
        line_id = p.createCollisionShape(p.GEOM_BOX, halfExtents=[length/2, thickness/2, 0.01])
        line_body = p.createMultiBody(baseMass=0,
                          baseCollisionShapeIndex=line_id,
                          basePosition=[(start_pos[0] + end_pos[0])/2, (start_pos[1] + end_pos[1])/2, 0.01],
                          baseOrientation=p.getQuaternionFromEuler([0, 0, orientation]))
        p.changeVisualShape(line_body, -1, rgbaColor=[1, 1, 1, 1])  # White line

    def add_circle(self, center_pos, radius, line_thickness):
        # Create circle using multiple short lines to approximate a circle
        num_segments = 18  # Reduced number of segments to simplify rendering
        angle_increment = 2 * np.pi / num_segments
        for i in range(num_segments):
            start_pos = [
                center_pos[0] + radius * np.cos(i * angle_increment),
                center_pos[1] + radius * np.sin(i * angle_increment)
            ]
            end_pos = [
                center_pos[0] + radius * np.cos((i + 1) * angle_increment),
                center_pos[1] + radius * np.sin((i + 1) * angle_increment)
            ]
            self.add_line(start_pos, end_pos, line_thickness)

    def create_goal(self, position, goal_width=7.32, goal_height=2.44):
        # Create goal posts and crossbar using box shapes
        post_thickness = 0.1

        # Left post
        p.loadURDF("cube.urdf", basePosition=[position[0], position[1] - goal_width / 2, goal_height / 2], globalScaling=post_thickness)
        # Right post
        p.loadURDF("cube.urdf", basePosition=[position[0], position[1] + goal_width / 2, goal_height / 2], globalScaling=post_thickness)
        # Crossbar
        p.loadURDF("cube.urdf", basePosition=[position[0], position[1], goal_height], globalScaling=post_thickness)



In [ ]:

class ViewEnvironment:
    def __init__(self):
        # Connect to PyBullet in GUI mode
        self.physics_client = p.connect(p.GUI)
        self.env = Environment()  # Initialize the environment

    def view(self, duration=10):
        # Run the simulation for a shorter duration to view the environment
        for _ in range(int(duration * 240)):  # Duration in seconds, assuming 240 steps per second
            p.stepSimulation()
            # Reduce sleep time to speed up if real-time observation isn't necessary
            time.sleep(1. / 480.)  # Run faster than real-time
        p.disconnect()


# Create and view the environment
viewer = ViewEnvironment()
viewer.view(duration=5)  # Reduced duration to 5 seconds for quick viewing


In [ ]:
# Agent class for controlling the humanoid agent
class Agent:
    def __init__(self):
        self.loadAgent(urdf_file="URDFs/humanoid.urdf", start_pos=[0, 0, 2], start_orientation=[0, 0, 0])

    def loadAgent(self, urdf_file, start_pos, start_orientation):
        # Load agents and get joint indices
        self.humanoid = p.loadURDF(urdf_file, start_pos, p.getQuaternionFromEuler(start_orientation))
        self.joint_indices = range(p.getNumJoints(self.humanoid))

    def moveJoint(self, joint_index):
        # Move agent's joints with random torque
        for joint_index in self.joint_indices:
            p.setJointMotorControl2(self.humanoid, joint_index, p.TORQUE_CONTROL, force=np.random.uniform(-100, 100))
        p.stepSimulation()

In [ ]:
# Create and view the environment
viewer = ViewEnvironment()
viewer.view(duration=10)  # Set duration to 20 seconds for viewing

In [ ]:
# Initialize and run the simulation
# sim = Simulation()
# vid = sim.playVideo()
# vid